In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
import nltk
from nltk.corpus import stopwords
import re,string,unicodedata
from bs4 import BeautifulSoup
import tensorflow_hub as hub
import tensorflow as tf
import pickle
import fasttext
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score as acc
nltk.download('stopwords')
pd.set_option('display.max_colwidth', 200)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tryptophanv2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# read data
train_data = pd.read_csv("../Data/train.csv")
test_data = pd.read_csv("../Data/test.csv")

In [3]:
#Parse dataset -- In total : 28619 samples

# get stopwords
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

#Apply function on headline column
train_data['headline']=train_data['headline'].apply(denoise_text)
test_data['headline']=test_data['headline'].apply(denoise_text)

In [4]:
train_data.head()

,is_sarcastic,headline
0,1,'you better give dad good trade deal sorry!' shout angry trump boys phone employee local chinese restaurant
1,0,open letter graduation speakers
2,0,donald trump makes dubious claim inauguration singer jackie evancho
3,1,pope francis clarifies god one many immortal beings speak every day
4,1,woman geared complain work sidelined friend marital problems


In [5]:
# Create fast text formatted 
def generate_txt_file(data,file_name):
    for index, row in data.iterrows():
        line = '__label__' + str(row['is_sarcastic']) + ' ' + row['headline']+'\n'
        with open(f'../Data/{file_name}','a') as out:
            out.write(line)

In [6]:
# generate fast text formatted train and test data
generate_txt_file(train_data,'fastTextTrain.txt')
generate_txt_file(test_data,'fastTextTest.txt')

In [66]:
# Define model
model = fasttext.train_supervised(input="../Data/fastTextTrain.txt",epoch=35,wordNgrams=5)

In [67]:
# Get predicted probabilities and their corresponding labels
labels = []
probs = []
for headline in test_data['headline']:
    label,prob = model.predict(headline)
    labels.append(label)
    probs.append(prob)
    
probs = [float(x[0]) for x in probs]
labels = [int(x[0][9]) for x in labels]

In [68]:
# Print accuracy
acc(labels,test_data['is_sarcastic'].tolist())

0.8106219426974144

In [75]:
# get prediction probabilities for the class 1
probs_for_1 = []
for label,prob in zip(labels,probs):
    if label == 0:
        probs_for_1.append(1-prob)
    else:
        probs_for_1.append(prob)

In [76]:
# Dump prediction probabilities
with open('./probs_for_1.pkl','wb') as f:
    pickle.dump(probs_for_1,f)